In [ ]:
# 日本語フォント設定（共通モジュール）
import warnings
warnings.filterwarnings('ignore')

# よく使うライブラリを先に読み込む
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ウィジェットの有無を通知・フラグ化
try:
    import ipywidgets as widgets
    from IPython.display import display, HTML, clear_output
    WIDGETS_AVAILABLE = True
    print('✅ ipywidgets利用可能')
except Exception:
    WIDGETS_AVAILABLE = False
    print('⚠️ ipywidgets未インストール - 一部機能制限')

import font_setup
JP_FP = font_setup.setup_fonts(show_test=False)


In [2]:
# %% [markdown]
# # 2. データ読み込み関数

# %%
def load_data_optimized(file_path):
    """
    GPU/CPU両対応のデータ読み込み関数
    """
    print("📊 データ読み込み中...")
    
    # まずPandasで読み込み（エンコーディング自動判定）
    encodings = ['utf-8', 'cp932', 'shift_jis', 'latin1']
    df = None
    
    for encoding in encodings:
        try:
            df = pd.read_csv(file_path, encoding=encoding)
            print(f"✅ データ読み込み成功（{encoding}）: {len(df):,}行")
            break
        except UnicodeDecodeError:
            continue
        except Exception as e:
            print(f"⚠️ {encoding}でのエラー: {e}")
            continue
    
    if df is None:
        raise ValueError(f"❌ ファイル読み込み失敗: {file_path}")
    
    # GPU使用可能な場合はcuDFに変換
    if USE_CUDF and GPU_AVAILABLE:
        try:
            # 日付列を一時的に文字列として保持
            date_columns = []
            for col in df.columns:
                if '日付' in col or 'date' in col.lower():
                    date_columns.append(col)
                    df[col] = df[col].astype(str)
            
            # cuDFに変換
            gdf = cudf.from_pandas(df)
            print("✅ GPU（cuDF）形式に変換完了")
            
            # 日付列を戻す
            for col in date_columns:
                try:
                    gdf[col] = cudf.to_datetime(gdf[col])
                except:
                    print(f"⚠️ {col}の日付変換失敗")
            
            return gdf, True  # GPU dataframe, is_gpu flag
        except Exception as e:
            print(f"⚠️ GPU変換失敗: {e}")
            print("   CPUモードで継続します")
    
    return df, False  # CPU dataframe, is_gpu flag

In [3]:
# データパスの設定（複数のパスを試す）
possible_paths = [
    'output/06_final_enriched_20250701_20250930.csv',  # 最新の特徴量付きデータ
    'output/06_cleaned_20250701_20250930.csv',  # クリーニング済みデータ
    'output/06_converted_20250701_20250930.csv'  # 変換済みデータ
]

df = None
is_gpu_df = False

for file_path in possible_paths:
    if Path(file_path).exists():
        df, is_gpu_df = load_data_optimized(file_path)
        print(f"📊 使用ファイル: {file_path}")
        print(f"📊 使用モード: {'GPU (cuDF)' if is_gpu_df else 'CPU (pandas)'}")
        break
    else:
        print(f"⚠️ ファイルが見つかりません: {file_path}")

if df is None:
    print("❌ データファイルが見つかりません")
    print("以下のいずれかの場所にCSVファイルを配置してください:")
    for path in possible_paths:
        print(f"  - {path}")
    raise FileNotFoundError("データファイルが見つかりません")

📊 データ読み込み中...
✅ データ読み込み成功（utf-8）: 83,789行
✅ GPU（cuDF）形式に変換完了
📊 使用ファイル: output/06_final_enriched_20250701_20250930.csv
📊 使用モード: GPU (cuDF)


In [ ]:
# %% [markdown]
# # 6. cuML GPU加速モデリング + PyCaret統合

# %%
print("="*80)
print("🚀 cuML GPU加速モデリング")
print("="*80)

# cuMLとPyCaretのインポート
import cuml
import cupy as cp
from pycaret.regression import *

# GPUメモリ確認
try:
    import pynvml
    pynvml.nvmlInit()
    handle = pynvml.nvmlDeviceGetHandleByIndex(0)
    info = pynvml.nvmlDeviceGetMemoryInfo(handle)
    print(f"📊 GPU メモリ: {info.used/1e9:.1f}/{info.total/1e9:.1f} GB使用中")
except:
    pass

print("✅ cuML GPU加速モードで実行")

# GPU/CPU両対応のデータフレーム変換
if is_gpu_df:
    df_modeling = df.to_pandas()
    print("📊 GPU DataFrame → CPU DataFrame変換完了")
else:
    df_modeling = df.copy()

# 特徴量と目的変数の準備
print("\n📊 特徴量エンジニアリング中...")

df_modeling['年'] = df_modeling['日付'].dt.year
df_modeling['月'] = df_modeling['日付'].dt.month
df_modeling['日'] = df_modeling['日付'].dt.day
df_modeling['曜日'] = df_modeling['日付'].dt.dayofweek
df_modeling['週'] = df_modeling['日付'].dt.isocalendar().week

exclude_cols = ['日付', '商品名', '店舗', 'フェイスくくり大分類', 'フェイスくくり中分類',
                'フェイスくくり小分類', '売上金額']
target = '売上数量'

feature_cols = [col for col in df_modeling.select_dtypes(include=[np.number]).columns
                if col not in exclude_cols and col != target]

print(f"\n使用する特徴量: {len(feature_cols)}個")
print(f"目的変数: {target}")

modeling_data = df_modeling[feature_cols + [target]].copy()
modeling_data = modeling_data.dropna()
print(f"\nモデリングデータ: {len(modeling_data):,}行")

# データ分割
train_size = int(len(modeling_data) * 0.8)
train_data = modeling_data.iloc[:train_size].copy()
test_data = modeling_data.iloc[train_size:].copy()

print(f"訓練データ: {len(train_data):,}行")
print(f"テストデータ: {len(test_data):,}行")

# %%
print("="*80)
print("🚀 cuML GPUモデル群による高速比較")
print("="*80)

# GPU用データの準備
X_train_gpu = cp.array(train_data[feature_cols].values, dtype=cp.float32)
y_train_gpu = cp.array(train_data[target].values, dtype=cp.float32)
X_test_gpu = cp.array(test_data[feature_cols].values, dtype=cp.float32)
y_test = test_data[target].values

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

results = []

print("\n🔥 cuML GPUモデルをトレーニング中...")

# 1. cuML Random Forest (GPU)
print("\n1️⃣ cuML Random Forest (GPU)")
from cuml.ensemble import RandomForestRegressor as cuRF
model_rf = cuRF(n_estimators=100, max_depth=15, n_bins=128, random_state=123)
model_rf.fit(X_train_gpu, y_train_gpu)
pred_rf = model_rf.predict(X_test_gpu).get()
mae_rf = mean_absolute_error(y_test, pred_rf)
r2_rf = r2_score(y_test, pred_rf)
results.append(('cuML Random Forest (GPU)', mae_rf, r2_rf, model_rf))
print(f"   MAE: {mae_rf:.2f} | R²: {r2_rf:.4f}")

# 2. cuML Linear Regression (GPU)
print("\n2️⃣ cuML Linear Regression (GPU)")
from cuml.linear_model import LinearRegression as cuLR
model_lr = cuLR()
model_lr.fit(X_train_gpu, y_train_gpu)
pred_lr = model_lr.predict(X_test_gpu).get()
mae_lr = mean_absolute_error(y_test, pred_lr)
r2_lr = r2_score(y_test, pred_lr)
results.append(('cuML Linear Regression (GPU)', mae_lr, r2_lr, model_lr))
print(f"   MAE: {mae_lr:.2f} | R²: {r2_lr:.4f}")

# 3. cuML Ridge (GPU)
print("\n3️⃣ cuML Ridge (GPU)")
from cuml.linear_model import Ridge as cuRidge
model_ridge = cuRidge(alpha=1.0)
model_ridge.fit(X_train_gpu, y_train_gpu)
pred_ridge = model_ridge.predict(X_test_gpu).get()
mae_ridge = mean_absolute_error(y_test, pred_ridge)
r2_ridge = r2_score(y_test, pred_ridge)
results.append(('cuML Ridge (GPU)', mae_ridge, r2_ridge, model_ridge))
print(f"   MAE: {mae_ridge:.2f} | R²: {r2_ridge:.4f}")

# 4. cuML Lasso (GPU)
print("\n4️⃣ cuML Lasso (GPU)")
from cuml.linear_model import Lasso as cuLasso
model_lasso = cuLasso(alpha=0.1)
model_lasso.fit(X_train_gpu, y_train_gpu)
pred_lasso = model_lasso.predict(X_test_gpu).get()
mae_lasso = mean_absolute_error(y_test, pred_lasso)
r2_lasso = r2_score(y_test, pred_lasso)
results.append(('cuML Lasso (GPU)', mae_lasso, r2_lasso, model_lasso))
print(f"   MAE: {mae_lasso:.2f} | R²: {r2_lasso:.4f}")

# 5. cuML KNN (GPU)
print("\n5️⃣ cuML KNN (GPU)")
from cuml.neighbors import KNeighborsRegressor as cuKNN
model_knn = cuKNN(n_neighbors=5)
model_knn.fit(X_train_gpu, y_train_gpu)
pred_knn = model_knn.predict(X_test_gpu).get()
mae_knn = mean_absolute_error(y_test, pred_knn)
r2_knn = r2_score(y_test, pred_knn)
results.append(('cuML KNN (GPU)', mae_knn, r2_knn, model_knn))
print(f"   MAE: {mae_knn:.2f} | R²: {r2_knn:.4f}")

# 6. cuML SVR (GPU)
print("\n6️⃣ cuML SVR (GPU)")
from cuml.svm import SVR as cuSVR
model_svr = cuSVR(kernel='rbf', C=1.0)
model_svr.fit(X_train_gpu, y_train_gpu)
pred_svr = model_svr.predict(X_test_gpu).get()
mae_svr = mean_absolute_error(y_test, pred_svr)
r2_svr = r2_score(y_test, pred_svr)
results.append(('cuML SVR (GPU)', mae_svr, r2_svr, model_svr))
print(f"   MAE: {mae_svr:.2f} | R²: {r2_svr:.4f}")

print("\n" + "="*80)
print("✅ cuML GPUモデル比較完了")
print("="*80)

# 結果をソート
results.sort(key=lambda x: x[1])  # MAEでソート

print("\n【cuML GPUモデルランキング（MAE順）】")
for i, (name, mae, r2, model) in enumerate(results, 1):
    print(f"{i}. {name}")
    print(f"   MAE: {mae:.2f} | R²: {r2:.4f}")

# ベストモデルを選択
best_name, best_mae, best_r2, best_model = results[0]

# %%
print("="*80)
print("🔧 PyCaretによる追加モデル比較（CPU）")
print("="*80)

print("\ncuMLの結果と比較するため、PyCaretでも主要モデルを実行します...")
print("（高速処理のため主要モデルのみ）")

# PyCaretセットアップ
exp = setup(
    data=train_data,
    target=target,
    session_id=123,
    verbose=False,
    html=False,
    n_jobs=-1,
    normalize=True,
    transformation=True,
    polynomial_features=False,
    feature_selection=False,  # 高速化のため
    remove_multicollinearity=False
)

# 主要モデルのみ比較（高速・利用可能なモデルのみ）
print("\n📊 PyCaretモデル比較中...")
include_models = ['lr', 'ridge', 'lasso', 'rf', 'et', 'gbr', 'xgboost', 'lightgbm']
pycaret_models = compare_models(
    include=include_models,
    n_select=5,
    sort='MAE',
    verbose=False,
    errors='ignore'
)

print("\n✅ PyCaret比較完了")

# %%
print("="*80)
print("🏆 最終評価 - cuML GPU vs PyCaret")
print("="*80)

print(f"\n【cuML GPU ベストモデル】")
print(f"モデル: {best_name}")
print(f"MAE: {best_mae:.2f}")
print(f"R²: {best_r2:.4f}")

print(f"\n【PyCaret ベストモデル】")
pycaret_best = pycaret_models[0]
pycaret_pred = predict_model(pycaret_best, data=test_data)
pycaret_mae = mean_absolute_error(pycaret_pred[target], pycaret_pred['prediction_label'])
pycaret_r2 = r2_score(pycaret_pred[target], pycaret_pred['prediction_label'])
print(f"モデル: {pycaret_best.__class__.__name__}")
print(f"MAE: {pycaret_mae:.2f}")
print(f"R²: {pycaret_r2:.4f}")

# 最終的なベストモデルを決定
if best_mae < pycaret_mae:
    final_model = best_model
    final_model_name = best_name
    final_mae = best_mae
    final_r2 = best_r2
    is_cuml = True
    print(f"\n🏆 最終選択: {best_name} (cuML GPU版)")
else:
    final_model = pycaret_best
    final_model_name = pycaret_best.__class__.__name__
    final_mae = pycaret_mae
    final_r2 = pycaret_r2
    is_cuml = False
    print(f"\n🏆 最終選択: {pycaret_best.__class__.__name__} (PyCaret)")

# %%
print("="*80)
print("📊 最終モデル評価")
print("="*80)

if is_cuml:
    final_predictions = final_model.predict(X_test_gpu).get()
else:
    final_pred = predict_model(final_model, data=test_data)
    final_predictions = final_pred['prediction_label'].values

actual = test_data[target].values
predicted = final_predictions

mae = mean_absolute_error(actual, predicted)
rmse = np.sqrt(mean_squared_error(actual, predicted))
r2 = r2_score(actual, predicted)
from sklearn.metrics import mean_absolute_percentage_error
mape = mean_absolute_percentage_error(actual, predicted) * 100

print("\n【予測精度】")
print(f"  MAE (平均絶対誤差): {mae:.2f}個")
print(f"  RMSE (二乗平均平方根誤差): {rmse:.2f}個")
print(f"  R² (決定係数): {r2:.4f}")
print(f"  MAPE (平均絶対パーセント誤差): {mape:.2f}%")

print("\n【解釈】")
print(f"  ✓ 予測値は平均して実測値から{mae:.1f}個の誤差がある")
print(f"  ✓ モデルは売上変動の{r2*100:.1f}%を説明できている")
print(f"  ✓ 予測誤差は平均{mape:.1f}%")

# 実測値 vs 予測値のプロット
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

axes[0].scatter(actual, predicted, alpha=0.5, s=20, color='steelblue')
axes[0].plot([actual.min(), actual.max()], [actual.min(), actual.max()],
             'r--', linewidth=2, label='完璧な予測')
axes[0].set_title(f'実測値 vs 予測値 (R²={r2:.4f}, fontproperties=JP_FP)', fontsize=14)
axes[0].set_xlabel('実測値（個）', fontsize=11, fontproperties=JP_FP)
axes[0].set_ylabel('予測値（個）', fontsize=11, fontproperties=JP_FP)
axes[0].legend(fontsize=11, prop=JP_FP)
axes[0].grid(True, alpha=0.3)

residuals = actual - predicted
axes[1].scatter(predicted, residuals, alpha=0.5, s=20, color='coral')
axes[1].axhline(y=0, color='black', linestyle='--', linewidth=2)
axes[1].set_title('残差プロット', fontsize=14, fontproperties=JP_FP)
axes[1].set_xlabel('予測値（個）', fontsize=11, fontproperties=JP_FP)
axes[1].set_ylabel('残差（実測値 - 予測値）', fontsize=11, fontproperties=JP_FP)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# %%
print("="*80)
print("💾 モデル保存")
print("="*80)

model_path = 'output/pycaret_models/best_sales_prediction_model'
Path('output/pycaret_models').mkdir(parents=True, exist_ok=True)

if is_cuml:
    # cuMLモデルの保存
    import pickle
    with open(f'{model_path}_cuml.pkl', 'wb') as f:
        pickle.dump(final_model, f)
    print(f"\n✅ cuML GPUモデルを保存: {model_path}_cuml.pkl")
else:
    # PyCaretモデルの保存
    save_model(final_model, model_path)
    print(f"\n✅ PyCaretモデルを保存: {model_path}.pkl")

# モデル情報の保存
model_info = {
    'model_name': final_model_name,
    'model_type': 'cuML_GPU' if is_cuml else 'PyCaret',
    'mae': float(mae),
    'rmse': float(rmse),
    'r2': float(r2),
    'mape': float(mape),
    'train_size': len(train_data),
    'test_size': len(test_data),
    'features': feature_cols,
    'target': target,
    'gpu_accelerated': True,
    'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
}

import json
with open('output/pycaret_models/model_info.json', 'w', encoding='utf-8') as f:
    json.dump(model_info, f, ensure_ascii=False, indent=2)

print(f"✅ モデル情報を保存しました")

# %%
print("="*80)
print("🚀 予測実行デモ")
print("="*80)

demo_data = test_data.head(10).copy()

if is_cuml:
    X_demo_gpu = cp.array(demo_data[feature_cols].values, dtype=cp.float32)
    demo_pred = final_model.predict(X_demo_gpu).get()
else:
    demo_predictions = predict_model(final_model, data=demo_data)
    demo_pred = demo_predictions['prediction_label'].values

result_df = pd.DataFrame({
    '実測値': demo_data[target].values,
    '予測値': demo_pred.round(0),
    '誤差': (demo_pred - demo_data[target].values).round(1),
    '誤差率(%)': ((demo_pred - demo_data[target].values) / demo_data[target].values * 100).round(1)
})

print("\n【予測デモ（上位10件）】")
print(result_df.to_string(index=False))

print("\n" + "="*80)
print("✅ GPU加速モデリング完了")
print("="*80)
print(f"\n📊 最終モデル: {final_model_name}")
print(f"🖥️ 実行環境: {'cuML GPU加速' if is_cuml else 'PyCaret'}")
print(f"📈 予測精度 (MAE): {mae:.2f}個")
print(f"📈 決定係数 (R²): {r2:.4f}")
print(f"💾 保存先: {model_path}{'_cuml' if is_cuml else ''}.pkl")


🚀 cuML GPU加速モデリング
📊 GPU メモリ: 4.2/12.9 GB使用中
✅ cuML GPU加速モードで実行
📊 GPU DataFrame → CPU DataFrame変換完了

📊 特徴量エンジニアリング中...

使用する特徴量: 123個
目的変数: 売上数量

モデリングデータ: 71,485行
訓練データ: 57,188行
テストデータ: 14,297行
🚀 cuML GPUモデル群による高速比較

🔥 cuML GPUモデルをトレーニング中...

1️⃣ cuML Random Forest (GPU)
   MAE: 6.21 | R²: -0.1203

2️⃣ cuML Linear Regression (GPU)
   MAE: 14.07 | R²: -2.8816

3️⃣ cuML Ridge (GPU)
   MAE: 9.41 | R²: -1.1712

4️⃣ cuML Lasso (GPU)
   MAE: 4.68 | R²: 0.0077

5️⃣ cuML KNN (GPU)
   MAE: 5.49 | R²: -0.2199

6️⃣ cuML SVR (GPU)
   MAE: 4.07 | R²: -0.0950

✅ cuML GPUモデル比較完了

【cuML GPUモデルランキング（MAE順）】
1. cuML SVR (GPU)
   MAE: 4.07 | R²: -0.0950
2. cuML Lasso (GPU)
   MAE: 4.68 | R²: 0.0077
3. cuML KNN (GPU)
   MAE: 5.49 | R²: -0.2199
4. cuML Random Forest (GPU)
   MAE: 6.21 | R²: -0.1203
5. cuML Ridge (GPU)
   MAE: 9.41 | R²: -1.1712
6. cuML Linear Regression (GPU)
   MAE: 14.07 | R²: -2.8816
🔧 PyCaretによる追加モデル比較（CPU）

cuMLの結果と比較するため、PyCaretでも主要モデルを実行します...
（高速処理のため主要モデルのみ）

📊 PyCaretモデル比較中...
